In [3]:
!pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.8/251.8 kB 6.0 MB/s eta 0:00:00


In [4]:
import pandas as pd
import sklearn
import numpy as np
import tensorflow as tf
import keras
import seaborn as sns
import fastai

In [5]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [6]:
client_credentials_manager = SpotifyClientCredentials(client_id="5b7651013fa24a4f8a143d1d76b48e8c", client_secret="ca39542e3ba64ea88d7d62a196c07db2")
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [7]:
import json
f = open('challenge_set.json')

In [8]:
data = json.load(f)

In [9]:
len(data["playlists"])

10000

In [10]:
for i in data["playlists"]:
  if not i["tracks"]:
    data["playlists"].remove(i)

In [11]:
len(data["playlists"])

9500

In [12]:
plays = []

for i in data["playlists"]:
  if len(i['tracks']) > 10:
    uriList = []
    aUriList = []
    for j in range(10):
      tUri = i['tracks'][j]['track_uri']
      aUri = i['tracks'][j]['artist_uri']
      uriList.append(tUri)
      aUriList.append(aUri)
    curTracks = sp.tracks(uriList)
    trackFeats = sp.audio_features(uriList)
    trackArtists = sp.artists(aUriList)
    trackList = []
    for j in range(10):
      trackDict = {}
      trackDict['track_name'] = i['tracks'][j]['track_name']
      trackFeat = trackFeats[j]
      trackFromURI = curTracks['tracks'][j]
      trackArtist = trackArtists['artists'][j]
      trackDict['danceability'] = trackFeat['danceability']
      trackDict['energy'] = trackFeat['energy']
      trackDict['tempo'] = trackFeat['tempo']
      trackDict['duration_ms'] = trackFeat['duration_ms']
      if trackArtist['genres']:
        trackDict['genre'] = trackArtist['genres'][0]
      if trackFromURI['album']['release_date']:
        trackDict['release_year'] = trackFromURI['album']['release_date'][0:4]
      trackList.append(trackDict)
    plays.append(trackList)
    if len(plays) >= 10:
      break

In [13]:
print(plays[0])

[{'track_name': 'Before I Let Go (feat. Frankie Beverly) [2004 - Remastered]', 'danceability': 0.783, 'energy': 0.563, 'tempo': 103.069, 'duration_ms': 236107, 'genre': 'classic soul', 'release_year': '2004'}, {'track_name': 'Me and Mrs. Jones', 'danceability': 0.303, 'energy': 0.555, 'tempo': 67.322, 'duration_ms': 285760, 'genre': 'classic soul', 'release_year': '1972'}, {'track_name': 'Love TKO (Re-Recorded / Remastered)', 'danceability': 0.683, 'energy': 0.579, 'tempo': 145.064, 'duration_ms': 281983, 'genre': 'classic soul', 'release_year': '2004'}, {'track_name': 'Turn off the Lights', 'danceability': 0.63, 'energy': 0.494, 'tempo': 118.501, 'duration_ms': 354853, 'genre': 'classic soul', 'release_year': '1977'}, {'track_name': 'Sexual Healing', 'danceability': 0.805, 'energy': 0.602, 'tempo': 94.382, 'duration_ms': 238600, 'genre': 'classic soul', 'release_year': '1982'}, {'track_name': 'After The Dance - Vocal', 'danceability': 0.734, 'energy': 0.49, 'tempo': 109.738, 'duration

In [14]:
lookTrack = False
currentTrack = False

for i in plays:
  for j in range(9):
    numSim = 0
    numCurrent = 0
    numLook = 0
    for k in data["playlists"]:
      if k["tracks"]:
        if any(d["track_name"] == i[-1]["track_name"] for d in k["tracks"]):
          lookTrack = True
          numLook += 1
        if any(d["track_name"] == i[j]["track_name"] for d in k["tracks"]):
          currentTrack = True
          numCurrent += 1
        if lookTrack and currentTrack:
            numSim += 1
        lookTrack = False
        currentTrack = False
    i[j]["sorenson"] = (2*numSim)/(numCurrent+numLook)
    i[j]["num_similar"] = numSim

In [15]:
dfList = []

for i in plays:
  longPlay = []
  for j in i:
    for k, v in j.items():
      longPlay.append(v)
  dfList.append(longPlay)

for i in range(len(dfList)):
  dfList[i].append(1)

for i in dfList:
  for j in i:
    print(j)

Before I Let Go (feat. Frankie Beverly) [2004 - Remastered]
0.783
0.563
103.069
236107
classic soul
2004
0.06896551724137931
1
Me and Mrs. Jones
0.303
0.555
67.322
285760
classic soul
1972
0.06666666666666667
1
Love TKO (Re-Recorded / Remastered)
0.683
0.579
145.064
281983
classic soul
2004
0.07142857142857142
1
Turn off the Lights
0.63
0.494
118.501
354853
classic soul
1977
0.06451612903225806
1
Sexual Healing
0.805
0.602
94.382
238600
classic soul
1982
0.03773584905660377
1
After The Dance - Vocal
0.734
0.49
109.738
279760
classic soul
1976
0.06666666666666667
1
Soon I'll Be Loving You Again
0.596
0.613
95.039
192773
classic soul
1976
0.07142857142857142
1
Your Precious Love
0.429
0.265
203.018
187267
classic soul
1967
0.06666666666666667
1
Save The Children
0.307
0.688
106.626
242587
classic soul
1971
0.07142857142857142
1
I Want You
0.515
0.793
100.551
274400
classic soul
1976
1
Make You Feel My Love
0.325
0.18
72.416
212040
british soul
2008
0.10526315789473684
5
Free Fallin' - Li

In [16]:
df = pd.DataFrame(data = dfList, columns=['title0', 'danceability0', 'energy0', 'tempo0', 'duration_ms0', 'genre0', 'release_year0', 'num_similar0', "sorenson0",
                           'title1', 'danceability1', 'energy1', 'tempo1', 'duration_ms1', 'genre1', 'release_year1', 'num_similar1', "sorenson1",
                           'title2', 'danceability2', 'energy2', 'tempo2', 'duration_ms2', 'genre2', 'release_year2', 'num_similar2', "sorenson2",
                           'title3', 'danceability3', 'energy3', 'tempo3', 'duration_ms3', 'genre3', 'release_year3', 'num_similar3', "sorenson3",
                           'title4', 'danceability4', 'energy4', 'tempo4', 'duration_ms4', 'genre4', 'release_year4', 'num_similar4', "sorenson4",
                           'title5', 'danceability5', 'energy5', 'tempo5', 'duration_ms5', 'genre5', 'release_year5', 'num_similar5', "sorenson5",
                           'title6', 'danceability6', 'energy6', 'tempo6', 'duration_ms6', 'genre6', 'release_year6', 'num_similar6', "sorenson6",
                           'title7', 'danceability7', 'energy7', 'tempo7', 'duration_ms7', 'genre7', 'release_year7', 'num_similar7', "sorenson7",
                           'title8', 'danceability8', 'energy8', 'tempo8', 'duration_ms8', 'genre8', 'release_year8', 'num_similar8', "sorenson8",
                           'title9', 'danceability9', 'energy9', 'tempo9', 'duration_ms9', 'genre9', 'release_year9', 'alike_song'])

In [17]:
df.head(10)

,title0,danceability0,energy0,tempo0,duration_ms0,genre0,release_year0,num_similar0,sorenson0,title1,...,num_similar8,sorenson8,title9,danceability9,energy9,tempo9,duration_ms9,genre9,release_year9,alike_song
0,Before I Let Go (feat. Frankie Beverly) [2004 ...,0.783,0.563,103.069,236107,classic soul,2004,0.068966,1,Me and Mrs. Jones,...,0.071429,1,I Want You,0.515,0.793,100.551,274400,classic soul,1976,1
1,Make You Feel My Love,0.325,0.180,72.416,212040,british soul,2008,0.105263,5,Free Fallin' - Live at the Nokia Theatre,...,0.184615,6,Your Body Is a Wonderland,0.647,0.729,94.010,249627,neo mellow,2001,1
2,Stressed Out,0.734,0.637,169.977,202333,modern rock,2015,0.029412,2,Some Boys,...,0.074074,1,Flawless,0.568,0.871,119.990,246493,modern alternative rock,2013,1
3,"Circle Of Life - From ""The Lion King""/Soundtrack",0.402,0.391,83.789,239147,movie tunes,1994,0.333333,6,"I Just Can't Wait to Be King - From ""The Lion ...",...,0.387097,6,"When Will My Life Begin - From ""Tangled""/Sound...",0.673,0.420,108.066,152333,dance pop,2010,1
4,Click Click Boom,0.609,0.935,95.957,252400,alternative metal,2001,0.090909,2,Alive - New Album/Chris Lord-Alge Mix Version,...,0.333333,10,Headstrong,0.492,0.896,184.981,285570,alternative metal,2002,1
5,Throne,0.425,0.882,144.117,190920,melodic metalcore,2015,0.029851,1,Drown - New,...,0.293578,16,American Idiot,0.380,0.988,186.113,176346,modern rock,2004,1
6,Starboy,0.681,0.594,186.054,230453,canadian contemporary r&b,2016,0.074534,6,Let Me Love You,...,0.083333,2,Perfect Strangers,0.739,0.833,117.990,195313,pop,2017,1
7,Lord Pretty Flacko Jodye 2 (LPFJ2),0.485,0.720,207.982,126960,east coast hip hop,2015,0.070796,4,Electric Body,...,0.270270,15,Fight Night,0.874,0.706,89.961,216248,atl hip hop,2014,1
8,All I Do Is Win,0.549,0.791,150.093,227293,hip hop,2010,0.024390,1,Young Dumb & Broke,...,0.058824,2,Aftergold,0.602,0.901,94.570,191007,electropop,2016,1
9,If I Told You,0.532,0.640,84.029,212320,black americana,2017,0.058824,1,Southern Land,...,0.153846,1,The High Life (feat. Chase Rice),0.553,0.846,149.985,245354,contemporary country,2014,1


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 89 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title0         10 non-null     object 
 1   danceability0  10 non-null     float64
 2   energy0        10 non-null     float64
 3   tempo0         10 non-null     float64
 4   duration_ms0   10 non-null     int64  
 5   genre0         10 non-null     object 
 6   release_year0  10 non-null     object 
 7   num_similar0   10 non-null     float64
 8   sorenson0      10 non-null     int64  
 9   title1         10 non-null     object 
 10  danceability1  10 non-null     float64
 11  energy1        10 non-null     float64
 12  tempo1         10 non-null     float64
 13  duration_ms1   10 non-null     int64  
 14  genre1         10 non-null     object 
 15  release_year1  10 non-null     object 
 16  num_similar1   10 non-null     float64
 17  sorenson1      10 non-null     int64  
 18  title2       

In [19]:
titles = ["title0", "title1", "title2", "title3", "title4", "title5", "title6", "title7", "title8", "title9"]
years = ["release_year0", "release_year1", "release_year2", "release_year3", "release_year4", "release_year5", "release_year6", "release_year7", "release_year8", "release_year9"]
genres = ["genre0", "genre1", "genre2", "genre3", "genre4", "genre5", "genre6", "genre7", "genre8", "genre9"]

df = df.drop(columns=titles)

In [20]:
def fix_year(word):
    if not pd.isnull(word):
        return int(word)
    else:
        return word

for i in years:
    df[i] = df[i].apply(fix_year)

In [21]:
from sklearn.preprocessing import LabelEncoder

for i in genres:
  le = LabelEncoder()
  le.fit(df[i])
  df[i] = le.transform(df[i])


In [22]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['alike_song'])
y = df["alike_song"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [25]:
from keras.layers import Dense, Dropout

model = keras.Sequential()
model.add(Dense(70, input_dim = 78, activation = "relu"))
model.add(Dropout(0.8))
model.add(Dense(20, activation = "relu"))
model.add(Dropout(0.4))
model.add(Dense(7, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(3, activation = "relu"))
model.add(Dense(1, activation = "sigmoid"))

In [26]:
model.compile(loss = keras.losses.BinaryCrossentropy(), optimizer = keras.optimizers.Adam(learning_rate=1e-3), metrics = [keras.metrics.Accuracy(), keras.metrics.Precision()])
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 2390.2869 - accuracy: 0.5000 - precision_1: 1.0000
Epoch 2/10
1/1 [==============================] - 0s 25ms/step - loss: 1794.3282 - accuracy: 0.3750 - precision_1: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 22ms/step - loss: 2673.4644 - accuracy: 0.5000 - precision_1: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 20ms/step - loss: 0.1729 - accuracy: 0.7500 - precision_1: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 16ms/step - loss: 1575.5715 - accuracy: 0.5000 - precision_1: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 17ms/step - loss: 0.5180 - accuracy: 0.2500 - precision_1: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 16ms/step - loss: 0.6038 - accuracy: 0.1250 - precision_1: 1.0000
Epoch 8/10
1/1 [==============================] - 0s 17ms/step - loss: 0.4309 - accuracy: 0.3750 - precision_1: 1.0000
Epoch 9/10
1/1 [======================